# Twitter Data Collection & Analysis

In this lesson, we're going to learn how to analyze and explore Twitter data with the Python/command line tool [twarc](https://twarc-project.readthedocs.io/en/latest/). We're specifically going to work with [twarc2](https://twarc-project.readthedocs.io/en/latest/twarc2/), which is designed for version 2 of the Twitter API (released in 2020) and the Academic Research track of the Twitter API (released in 2021), which enables researchers to collect tweets from the entire Twitter archive for free.

Twarc was developed by a project called [Documenting the Now](https://www.docnow.io/). The DocNow team develops tools and ethical frameworks for social media research.

## Install and Import Libraries

Because twarc relies on Twitter's API, we need to apply for a Twitter developer account and create a Twitter application before we can use it.

If you haven't done so already, you need to install twarc and configure twarc with your bearer token and/or API keys.

In [ ]:
!pip install twarc

To configure twarc, you need to open a Terminal and type `twarc2 configure`, then enter your bearer token

Then we're going to import pandas

In [ ]:
import pandas as pd
pd.options.display.max_colwidth = 400
pd.options.display.max_columns = 90

## Search Queries & Privacy Concerns

To collect tweets from the Twitter API, we need to make queries, or requests for specific kinds of tweets — e.g., `twarc2 search *query*`. The simplest kind of query is a keyword search, such as the phrase "David Foster Wallace bro," which should return any tweet that contains all of these words in any order — `twarc2 search "David Foster Wallace bro"`.

There are many other operators that we can add to a query, which would allow us to collect tweets only from specific Twitter users or locations, or to only collect tweets that meet certain conditions, such as containing an image or being authored by a verified Twitter user. Here's an excerpted table of search operators taken from [Twitter's documentation](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#list) about how to build a search query. There are many other operators beyond those included in this table, and I recommend reading through [Twitter's entire web page on this subject](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#list).


| Search Operator             | Explanation                                                                                                                                                                                                                                                                                                                                                                                                                                         |
|:--------------------:|:----------------------------------------------------------------------------------------------:|
| keyword              | Matches a keyword within the body of a Tweet. `so sweet and so cold`                                                                                          
| "exact phrase match" | Matches the exact phrase within the body of a Tweet. `"so sweet and so cold" OR "plums in the icebox"`                                                                                              |
| - | Do NOT match a keyword or operator `baldwin -alec`, `walt whitman -bridge`                                                                                              |
| #                    | Matches any Tweet containing a recognized hashtag `#arthistory`        |                                                                             |
| from:, to:                | Matches any Tweet from or to a specific user. `from:KingJames` `to:KingJames`                                                                    |                                                                                                            |
| place:               | Matches Tweets tagged with the specified location or Twitter place ID. `place:"new york city" OR place:seattle`                                                                                            |
| is:reply, is:quote             | Returns only replies or quote tweets. `DFW bro is:reply` `David Foster Wallace bro is:quote`                                                                                                                               |
| is:verified          | Returns only Tweets whose authors are verified by Twitter.`DFW bro is:verified`                                                                                                                                |
| has:media           | Matches Tweets that contain a media object, such as a photo, GIF, or video, as determined by Twitter. `I Think You Should Leave has:media`                                                                                                                                |
| has:images, has:videos           | Matches Tweets that contain a recognized URL to an image. `i'm gonna tell my kinds that this was has:images`                                                                                    |
| has:geo              | Matches Tweets that have Tweet-specific geolocation data provided by the Twitter user.  `pyramids has:geo`                  

## Get Tweets (Standard Track)

To actually collect tweets and their associated metadata, we can use the command `twarc2 search` and insert a query.

Here we're going to search for any tweets that mention a word or phrase and were tweeted by verified accounts *in the past week*. By default, `twarc2 search` will use the standard track of the Twitter API, which only collects tweets from the past week.

In [ ]:
!twarc2 search "University of Washington is:verified"

The tweets and tweet metadata above are being printed to the notebook. But we want to save this information to a file so we can work with it.

To output Twitter data to a file, we can also include a filename with the ".jsonl" file extension, which stands for JSON lines, a special kind of JSON file.

In [ ]:
!twarc2 search "University of Washington is:verified" > tweets.jsonl

## Convert JSONL to CSV

To make our Twitter data easier to work with, we can convert our JSONL file to a CSV file with the [`twarc-csv`](https://pypi.org/project/twarc-csv/) plugin, which needs to be installed separately.

In [ ]:
!pip install twarc-csv

Once installed, we can use the plug-in from twarc2 with the input filename for the JSONL and a desired output filename for the CSV file.

In [ ]:
!twarc2 csv tweets.jsonl > tweets.csv

## Read in CSV

Now we're ready to explore the data!

To work with our tweet data, we can read in our CSV file with pandas and again parse the date column.

In [ ]:
tweets_df = pd.read_csv('tweets.csv',
                        parse_dates = ['created_at'])

In [ ]:
tweets_df

If we ask for a list of all the columns in the DataFrame, we can see that there are more than 90 columns here!

In [ ]:
tweets_df.columns

## Rename and Select Columns

To make the data more readable, we're going to rename a number of columns.

In [ ]:
tweets_df = tweets_df.rename(columns={'created_at': 'date',
                          'public_metrics.retweet_count': 'retweets', 
                          'author.username': 'username', 
                          'author.name': 'name',
                          'author.verified': 'verified', 
                          'public_metrics.like_count': 'likes', 
                          'public_metrics.quote_count': 'quotes', 
                          'public_metrics.reply_count': 'replies',
                           'author.description': 'user_bio'})

Then we're only going to select the columns that we're interested. Depending on your project and research question, you should change and customize these categories.

In [ ]:
tweets_df = tweets_df[['date', 'username', 'name', 'verified', 'text', 'retweets',
            'likes', 'replies',  'quotes', 'user_bio']]

Now we can view our more focused DataFrame!

In [ ]:
tweets_df

Try the whole thing at once and with an exact phrase match:

In [ ]:
!twarc2 search "\"University of Washington\" is:verified" > tweets.jsonl
!twarc2 csv tweets.jsonl > tweets.csv
tweets_df = pd.read_csv('tweets.csv',
                        parse_dates = ['created_at'])
tweets_df = tweets_df.rename(columns={'created_at': 'date',
                          'public_metrics.retweet_count': 'retweets', 
                          'author.username': 'username', 
                          'author.name': 'name',
                          'author.verified': 'verified', 
                          'public_metrics.like_count': 'likes', 
                          'public_metrics.quote_count': 'quotes', 
                          'public_metrics.reply_count': 'replies',
                           'author.description': 'user_bio'})
tweets_df = tweets_df[['date', 'username', 'name', 'verified', 'text', 'retweets',
           'likes', 'replies',  'quotes', 'user_bio']]
tweets_df

## Sort By Top Retweets

We can sort by number of retweets to see the most circulated tweets. Let's examine the top 5. The column is titled "retweets."

In [ ]:
# Sort top 5 tweets

## Sort By Date

We can sort from the earliest tweets to the latest tweets. Let's examine the earliest 5 tweets. The column is titled "date."

In [53]:
# Sort by top earliest tweets

## Get Tweet Counts

The [tweet counts API endpoint](https://twittercommunity.com/t/introducing-new-tweet-counts-endpoints-to-the-twitter-api-v2/155997) is a convenient feature of the v2 API (first introduced in 2021) that allows us to get a sense of how many tweets will be returned for a given query before we actually collect all the tweets that match the query.

The tweet counts API endpoint is perhaps even more useful for research projects that are primarily interested in tracking the volume of a Twitter conversation over time. In this case, tweet counts enable a researcher to retrieve this information in a way that's faster and easier than retrieving all tweets and relevant metadata.

To make an interactive plot, we're also going to install the package plotly.

In [ ]:
!pip install plotly

In [ ]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default='iframe'

To get tweet counts, we can use [`twarc2 counts`](https://twarc-project.readthedocs.io/en/latest/twarc2/#counts) followed by a search query.

We will also use the flag `--csv` because we want to output the data as a CSV and the flag `--granularity day` to get tweet counts per day (other options include `hour` and `minute` — you can see more in [twarc's documentation](https://twarc-project.readthedocs.io/en/latest/twarc2/#counts)).  Finally, we write the data to a CSV file.

In [ ]:
!twarc2 counts "University of Washington is:verified" --csv --granularity day > tweet-counts.csv

We can read in this CSV file with pandas, parse the date columns, and sort from earliest to latest. Then we can make a quick plot of tweets per day with [plotly](https://plotly.com/python/line-charts/) The code below is largely [borrowed from Ed Summers](https://github.com/edsu/notebooks/blob/master/Black%20Lives%20Matter%20Counts.ipynb). Thanks, Ed!

In [ ]:
# Code borrowed from Ed Summers
# https://github.com/edsu/notebooks/blob/master/Black%20Lives%20Matter%20Counts.ipynb

# Read in CSV as DataFrame
tweet_counts_df = pd.read_csv('tweet-counts.csv', parse_dates=['start', 'end'])

# Sort values by earliest date
tweet_counts_df = tweet_counts_df.sort_values('start')

figure = px.line(tweet_counts_df, x='start', y='day_count',
    labels={'start': 'Time', 'day_count': 'Tweets per Day'},
    title= 'Title Here'
)

figure.show()

With a plotly line chart, we can hover over points to see more information, and we can use the tool bar in the upper right corner to zoom or pan on different parts of the graph. We can also press the camera button to download an image of the graph at any pan or zoom level.

To return to the original view, double-click on the plot.